In [1]:
# Environment Setup & Imports
import ee
import geemap
import geopandas as gpd
import pandas as pd
import rasterio
import numpy as np
import matplotlib.pyplot as plt

folder = 'GEE_UHI_2013_2024'

In [2]:
# Init GEE
ee.Initialize(project='ee-uhi-analysis')

print("Environment ready. GEE Initialized.")


Environment ready. GEE Initialized.


In [3]:


# Create interactive map for AOI selection
Map = geemap.Map(center=[41.14, -104.82], zoom=12)
Map.add_basemap('SATELLITE')
Map

# Draw AOI: Click the map, use drawing tools (polygon/rectangle), export as GeoJSON


Map(center=[41.14, -104.82], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

In [4]:
aoi = Map.user_roi

if aoi:
    #  use it directly if able
    if isinstance(aoi, ee.Geometry):
        study_area = aoi
    elif isinstance(aoi, ee.Feature):
        study_area = aoi.geometry()
    elif isinstance(aoi, dict):  
        study_area = ee.Geometry(aoi["geometry"])
    else:
        raise ValueError("Unsupported AOI type")

    Map.addLayer(study_area, {}, "Selected Study Area")
    print("AOI successfully set as study_area.")
else:
    print("No geometry drawn. Please draw an AOI on the map.")


AOI successfully set as study_area.


In [5]:
def process_year(year, study_area):
    """Process and export NDVI + LST using Landsat 8 for any given year."""
    print(f"Processing year: {year} with Landsat 8")

    start_date = f'{year}-06-01'
    end_date = f'{year}-08-31'

    # Landsat 8 Collection
    imagery = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterBounds(study_area) \
        .filterDate(start_date, end_date) \
        .median()

    # NDVI & LST Calculation
    ndvi = imagery.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI').toFloat()
    lst = imagery.select('ST_B10').multiply(0.00341802).add(149.0).rename('LST').toFloat()
    export_image = ndvi.addBands(lst)

    # DEBUG: Image Info
    band_names = export_image.bandNames().getInfo()
    print(f"Bands: {band_names}")

    # Export Task
    task = ee.batch.Export.image.toDrive(
        image=export_image.clip(study_area),
        description=f'NDVI_LST_{year}',
        folder='GEE_UHI_2013_2024',  
        scale=30,
        region=study_area.bounds().getInfo()['coordinates'],
        fileFormat='GeoTIFF'
    )
    task.start()
    print(f"Export task started for year {year} | Task ID: {task.id}")


In [7]:
# Run processing for each year
years = range(2013, 2025)
for year in years:
    process_year(year, study_area)

Processing year: 2013 with Landsat 8
Bands: ['NDVI', 'LST']
Export task started for year 2013 | Task ID: 44KBVQRS3MLFR2IJCCH6RXM2
Processing year: 2014 with Landsat 8
Bands: ['NDVI', 'LST']
Export task started for year 2014 | Task ID: FIYYBZOGKQPS7LEQNHVY2O5C
Processing year: 2015 with Landsat 8
Bands: ['NDVI', 'LST']
Export task started for year 2015 | Task ID: JZ6LRIJBAIPRJQM5QBO5UAC7
Processing year: 2016 with Landsat 8
Bands: ['NDVI', 'LST']
Export task started for year 2016 | Task ID: XPY4E2YWDRYUAYUQOKDH7D32
Processing year: 2017 with Landsat 8
Bands: ['NDVI', 'LST']
Export task started for year 2017 | Task ID: 2ZRTKXU2V2FA46BFAMIU7UX4
Processing year: 2018 with Landsat 8
Bands: ['NDVI', 'LST']
Export task started for year 2018 | Task ID: NGJ3CHIWA5N2A2XXNAVGNWRW
Processing year: 2019 with Landsat 8
Bands: ['NDVI', 'LST']
Export task started for year 2019 | Task ID: ZOQLUV3I4WBNACW37LBBPIOT
Processing year: 2020 with Landsat 8
Bands: ['NDVI', 'LST']
Export task started for year 2

In [9]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
import os

# auth and create drive instance
gauth = GoogleAuth()
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=144780512349-929mvf5b2gn1qqpggq5dil6ptnpctt49.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=online&response_type=code

Authentication successful.


In [10]:
# export and output dir
export_folder_name = 'GEE_UHI_2013_2024'
local_download_dir = '../uhi-analysis/gee_exports'  

# Ensure local download directory exists
os.makedirs(local_download_dir, exist_ok=True)

# Get list of all files in Drive root
file_list = drive.ListFile({'q': "trashed=false"}).GetList()

# Find export folder ID
folder_id = None
for file in file_list:
    if file['title'] == export_folder_name and file['mimeType'] == 'application/vnd.google-apps.folder':
        folder_id = file['id']
        break

if not folder_id:
    print(f"ERROR Folder '{export_folder_name}' not found in Drive.")
else:
    print(f"INFO Export folder found: {export_folder_name} (ID: {folder_id})")

    # Get files in export folder
    export_files = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()

    # Download each
    for f in export_files:
        print(f"Downloading {f['title']}...")
        f.GetContentFile(os.path.join(local_download_dir, f['title']))
    print(f"All files downloaded to: {local_download_dir}")


INFO Export folder found: GEE_UHI_2013_2024 (ID: 1qBzvGF3q6MUmqN1rptnNIExSe-9KF72O)
All files downloaded to: ../uhi-analysis/gee_exports
